In [1]:
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews

### ID von App aus dem Playstore
https://play.google.com/store/apps/details?id=com.sec.android.app.shealth&gl=DE 
Die entsprechende ID lautet com.sec.android.app.shealth

In [2]:
apps = {
    'Samsung Health':'com.sec.android.app.shealth',
    'MyFitnessPal: Kalorienzähler':'com.myfitnesspal.android',
    'WW (Weight Watchers)':'com.weightwatchers.mobile'
}

In [15]:
def scrape_reviews(app_id):
    result, continuation_token = reviews(
        app_id,
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
        sort=Sort.MOST_RELEVANT, # defaults to Sort.NEWESTs
        count=100, # defaults to 100
    )
    return result   

In [16]:
reviews_df = pd.DataFrame()

In [17]:
for app_name, app_id in apps.items():
    result = scrape_reviews(app_id)
    app_review_df = pd.DataFrame(np.array(result),columns=['review'])
    app_review_df = app_review_df.join(pd.DataFrame(app_review_df.pop('review').to_list()))
    app_review_df['app_name'] = app_name
    reviews_df = pd.concat([reviews_df, app_review_df])

In [21]:
small_reviews_df = reviews_df[['app_name','at','content','score']]

In [22]:
small_reviews_df.to_csv('small_reviews.csv')